# Decision Tree: Implementation 

## Key Terminology
- **Root Node**: The starting point of the tree, representing the entire dataset.
- **Branch**: A split in the tree based on a feature and a threshold value.
- **Leaf Node**: The endpoint of a branch, representing a class label or prediction.

## What Needs to Be Decided?
1. **Split Feature**: Which feature to split on at the current node.
2. **Split Point**: The threshold or value to use for the split.
3. **When to Stop Splitting**: Based on predefined stopping criteria.

## Training (Building the Tree)
1. **Calculate Information Gain**:
   - Compute the **information gain** for all possible splits at a node.
   - Information gain formula:
     $$
     \text{Information Gain} = H(\text{parent}) - \left( \frac{n_{\text{left}}}{n} H(\text{left}) + \frac{n_{\text{right}}}{n} H(\text{right}) \right)
     $$
   - Where \( H \) is the entropy:
     $$
     H(y) = - \sum p(x) \cdot \log(p(x))
     $$
     $$
     p(x) = \frac{\text{\# of samples in class } x}{\text{total samples}}
     $$

2. **Split the Node**: - Divide the dataset using the feature and threshold with the highest information gain.
3. **Recursive Splitting**:- Repeat the process for all created branches.
4. **Stop When Criteria Are Met**:
   - Maximum depth is reached.
   - Minimum number of samples in a node is less than the threshold.
   - Minimum impurity decrease is achieved.

## Inference Time (Testing)
Given a new data point:
1. **Follow the Tree**: - Traverse the tree by evaluating splits at each node until a leaf node is reached.
2. **Return the Class Label**: - If the leaf node is **pure**, return its class label.- If the leaf node is **not pure**, return the majority class label.

## Stopping Criteria
1. **Maximum Depth**: The tree stops growing after reaching the specified depth.
2. **Minimum Samples per Node**: Splitting stops if the number of samples in a node is below a threshold.
3. **Minimum Impurity Decrease**: The tree stops splitting if the decrease in impurity is below a certain value.

## Summary of Key Steps
1. **Root Node**: Represents the entire dataset.
2. **Branch**: Represents a split in the data.
3. **Leaf Node**: Represents the class label for a subset of data.

**Core Formula**:
$$
\text{Information Gain} = H(\text{parent}) - \left( \frac{n_{\text{left}}}{n} H(\text{left}) + \frac{n_{\text{right}}}{n} H(\text{right}) \right)
$$


In [31]:
import numpy as np
from collections import Counter

class Node:
    #node consists of feature, which threshold it is split on, child node left and right childe 
    # node and if it is a leaf value of the leaft node i.e class of the leaf node
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    #if value is not none then it is a leaf node will be appened inthe training state and used for interference purpose    
    def is_leaf_node(self):
        return self.value is not None



'''
grow tree 
best split
information gain
entropy 
'''

class DecisionTree:
    #splitting and stopping critieria - min samples in the node to split , max layer depth of the tree, total number of features in the tree both splitting and 
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        

In [37]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

data  = datasets.load_breast_cancer()
X,y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X,y,test_size=0.2,random_state=1234
)


clf = DecisionTree(max_depth=10)

clf.fit(X_train,y_train)
pred = clf.predict(X_test)

#569 samples and #30 features 
#array([0, 1]) classes 
def accuracy(y_test,y_pred):
    return np.sum(y_test==y_pred)/len(y_test)
acc = accuracy(y_test=y_test,y_pred= pred)
print(acc)





0.9210526315789473
